In [69]:
import json
import numpy as np
import pylab as pl
import math
import time

class OnSource:
    
    def __init__(self, N_A, N_g, N_f, t0, tf, T, b, D, trials, A0=1, Af=50, g0=0, gf=2, F0=90, Ff=110, N_t=1000):
        print('very start...')
        """
        METHOD:

        PARAMETERS:
        -----------
        N_A: divisions in amplitude
        N_g:  divisions in gamma
        N_f: divisions in frequency
        t0 , tf: start and end time of injected signal, FIX!!!!!!!!
        T: length of total time series, in units of time
        b: noise amplitude
        D: foreground/background interval length
        trials: number of random data sets generated
        A0, Af : amplitude lower/upper bounds
        g0, gf : gamma lower/upper bounds
        F0, ff : frequency lower/upper bounds
        N_t= divisions in total time series 
        

        OUTPUT: Returns global maximum values for given ranges, and produces txt
        value containing all of them
        """
        print('initalizing...')
        ## initalizes the arrays which span parameter space
        ## and their respective lengths
        A_RANGE=np.linspace(A0,Af,N_A)
        A_LEN=len(A_RANGE)
        G_RANGE=np.linspace(g0,gf,N_g)
        G_LEN=len(G_RANGE)
        F_RANGE=np.linspace(F0,Ff,N_f)
        F_LEN=len(F_RANGE)
        
        
        ## Might need to fix this, error could propagate? : Constructs timestep resolution 
        self.dt=T/N_t
        
        print('building p-space...')
        ## constructs all templates which correspond
        ## to points in the parameter space
        self.TEMPLATES_AGF=[ self.template( A, f, g, tf-t0, self.dt) for A in A_RANGE 
                       for g in G_RANGE for f in F_RANGE]
        print('done p-space...')
        
        ## fix this, needs stuff from 'noise':
        ## constructs time-series length, foreground/background region
        ## lengths, and position of t0 in terms of said lengths
        L=len(np.linspace(0,T,N_t))
        dN=math.floor(2*D/(T/N_t))
        dNt0=math.floor(t0/2*D)
        
        
        ## constructs time range to pick injected signal start
        ## time from/ corresponding length 
        t_RANGE=np.linspace(0,T-(tf-t0),int(N_t*(1-((tf-t0)/(T)))))
        t_LEN=len(t_RANGE)

        ##constructs received data functions'data' , or d_i(t)
        ## finds their convolutions/ chi-squareds throughout
        ## the provided parameter space 
        self.noise=[]
        self.CRS_COR=[]
        self.CHI_SQR=[]
        
        ltri=trials
        ltemp=len(self.TEMPLATES_AGF)
        print(str(ltri)+'*'+str(ltemp)+'='+str(ltri*ltemp))
        
        ii=0
        jj=0
        
        for i in range(trials):
            print('trial '+str(ii)+':'+str(ltri))
            ii+=1
            
            A_RAN=np.random.randint(0,A_LEN)
            G_RAN=np.random.randint(0,G_LEN)
            F_RAN=np.random.randint(0,F_LEN)
            t_RAN=np.random.randint(0,t_LEN)
            print(str(A_RAN)+":"+str(G_RAN)+":"+str(F_RAN)+":"+str(t_RAN))
            
            ## generates data
            dt, T_full, data=self.waveform(F_RANGE[F_RAN],A_RANGE[A_RAN]
                                           ,b,t_RAN,t_RAN+(tf-t0),d_end_t=T,gamma=G_RANGE[G_RAN],N=N_t)
            self.noise.append(data)
            
            print('starting template loop...')
            ## performs base static calculation accross parameter space
            for template in self.TEMPLATES_AGF:
                if (jj%10==0):
                    print('trial '+str(jj)+':'+str(ltemp))
                    jj+=1
                
                print(template)
                CC_dh=self.CrossCorrelation(data, template, dt)
                self.CRS_COR.append(CC_dh)
                                            
                CS_dh=self.ChiSquare(data, template, dt)
                self.CHI_SQR.append(CS_dh)


    # produces data which contains random noise and an
    # injected signal 
    def waveform(self, f, A, b, t0, tend, d_end_t=None, gamma=0.0, phi0=0.0,
                 N=1000, verbose=False, seed_number=None, project_name=None):
        
        x=[f, A, b, t0, tend, d_end_t, gamma, phi0, N]
        print(x)
        
        # Conditional for noise duration
        # If the data-end time is supplied to be too small:
        assert t0 > 0, "Signal should start later than t=0"
        if (d_end_t is None) or (tend > d_end_t - 10):
            d_end_t = tend + 10
    
        T = np.linspace(t0, tend, N) # Time stamps of signal
        dt = np.mean(np.diff(T)) # figuring out the resolution of the series
    
        t = t0 # Initializing the time series at the start time
        t_minus = [] # To populate time stamps prior to the signal start
        while t >= 0: # Making sure that we reach all the way back to zero.
            t = t - dt
            t_minus.append(t)  # Create time spamps from (t0-dt) to 0

        t_minus = np.array(t_minus)[::-1]  # Reverse to be from 0 to t0
        t_minus = t_minus[t_minus >= 0]  # Eliminate numbers less than 0
    
        t_plus = np.arange(tend+dt, d_end_t, dt)  # Time stamps from (tend+dt) to d_end_t, in dt's
    
        T_full = np.hstack((t_minus, T, t_plus))  # Connect time stamps
    
        dev = np.std(np.diff(T_full))  # Standard deviation in dt's of T_full

        w = 2 * np.pi * f  
        y = A*np.sin(w*T + phi0)*np.exp(-gamma*(T-t0))
        
        
        print(y) ##########################################################################################
        
        
        # Padding of signal data
        y_minus = np.zeros_like(t_minus)
        y_plus = np.zeros_like(t_plus)
        y_full = np.hstack((y_minus, y, y_plus))

        noise = -b+2*b*np.random.random(len(T_full))  # Noise!
        
        d = noise + y_full  # Complete Data!
        
        return(dt, T_full, d)
    
    
    # Produces a cross corelation function given a data input
    # and a template in parameter space
    def CrossCorrelation(self, data, template, dt):
        ii = 0
        M = []
        
        while len(data[ii:]) >= len(template):
            M.append(np.sum((data[ii: len(template) + ii] * template)))
            ii+=1
        
        return M
                                            
    
    
    # Produces a template given a position in parameter space
    def template(self, A, f, gamma, duration, dt):
        t = np.arange(0, duration + dt, dt)
        w = 2 * np.pi * f
        return A * np.sin(w*t)*np.exp(-gamma*t)
    
    
    # Produces chi square at each "slide"
    def ChiSquare(self, data, template, dt):
        ii = 0
        C = []
        
        while len(data[ii:]) >= len(template):
            C.append(np.sum((data[ii: len(template) + ii] - template) ** 2))
            ii += 1
            
        return C


In [74]:
x=OnSource(1, 2, 2, 1, 3, 5, 0, 0.5, 1)

very start...
initalizing...
building p-space...
done p-space...
1*4=4
trial 0:1
0:0:1:206
[110.0, 1.0, 0, 206, 208, 5, 0.0, 0.0, 1000]
[-5.63536442e-12  9.82545587e-01  3.65550112e-01 -8.46544890e-01
 -6.80501991e-01  5.93368262e-01  9.01261046e-01 -2.58059569e-01
 -9.97270539e-01 -1.12968871e-01  9.55241158e-01  4.68360536e-01
 -7.80990468e-01 -7.58923291e-01  4.98637671e-01  9.44438405e-01
 -1.47265105e-01 -9.99227492e-01 -2.24491409e-01  9.15706829e-01
  5.65174567e-01 -7.05437067e-01 -8.27628139e-01  3.97523050e-01
  9.75524172e-01 -3.45852128e-02 -9.88391391e-01 -3.33139795e-01
  8.64448755e-01  6.54752698e-01 -6.20851995e-01 -8.85736908e-01
  2.91318963e-01  9.94120357e-01  7.85374716e-02 -9.64900969e-01
 -4.37523010e-01  8.02123193e-01  7.35948067e-01 -5.28318189e-01
 -9.32505637e-01  1.81385136e-01  9.99988874e-01  1.90654645e-01
 -9.29056973e-01 -5.36304643e-01  7.29528091e-01  8.07721134e-01
 -4.29020354e-01 -9.67335548e-01  6.91290446e-02  9.93054589e-01
  3.00330878e-01 -8

[ 0.00000000e+00 -3.09016994e-01  5.87785252e-01 -8.09016994e-01
  9.51056516e-01 -1.00000000e+00  9.51056516e-01 -8.09016994e-01
  5.87785252e-01 -3.09016994e-01 -2.20560220e-15  3.09016994e-01
 -5.87785252e-01  8.09016994e-01 -9.51056516e-01  1.00000000e+00
 -9.51056516e-01  8.09016994e-01 -5.87785252e-01  3.09016994e-01
  4.41120440e-15 -3.09016994e-01  5.87785252e-01 -8.09016994e-01
  9.51056516e-01 -1.00000000e+00  9.51056516e-01 -8.09016994e-01
  5.87785252e-01 -3.09016994e-01  4.88620758e-16  3.09016994e-01
 -5.87785252e-01  8.09016994e-01 -9.51056516e-01  1.00000000e+00
 -9.51056516e-01  8.09016994e-01 -5.87785252e-01  3.09016994e-01
  8.82240880e-15 -3.09016994e-01  5.87785252e-01 -8.09016994e-01
  9.51056516e-01 -1.00000000e+00  9.51056516e-01 -8.09016994e-01
  5.87785252e-01 -3.09016994e-01 -3.92258364e-15  3.09016994e-01
 -5.87785252e-01  8.09016994e-01 -9.51056516e-01  1.00000000e+00
 -9.51056516e-01  8.09016994e-01 -5.87785252e-01  3.09016994e-01
 -9.77241517e-16 -3.09016

[ 0.00000000e+00 -3.05942224e-01  5.76146324e-01 -7.85106929e-01
  9.13765057e-01 -9.51229425e-01  8.95671296e-01 -7.54322446e-01
  5.42594175e-01 -2.82420268e-01 -1.99571140e-15  2.76827972e-01
 -5.21318753e-01  7.10394126e-01 -8.26808815e-01  8.60707976e-01
 -8.10436903e-01  6.82539174e-01 -4.90959512e-01  2.55544426e-01
  3.61158870e-15 -2.50484307e-01  4.71708714e-01 -6.42791187e-01
  7.48127553e-01 -7.78800783e-01  7.33313635e-01 -6.17586984e-01
  4.44238537e-01 -2.31226159e-01  3.61979161e-16  2.26647574e-01
 -4.26819695e-01  5.81621518e-01 -6.76933804e-01  7.04688090e-01
 -6.63529616e-01  5.58815812e-01 -4.01963651e-01  2.09222080e-01
  5.91383747e-15 -2.05079206e-01  3.86202431e-01 -5.26272913e-01
  6.12515035e-01 -6.37628152e-01  6.00386425e-01 -5.05637457e-01
  3.63711752e-01 -1.89311967e-01 -2.37916724e-15  1.85563339e-01
 -3.49450410e-01  4.76191423e-01 -5.54226523e-01  5.76949810e-01
 -5.43252102e-01  4.57519691e-01 -3.29100003e-01  1.71296551e-01
 -5.36321516e-16 -1.67904

In [95]:
max(x.CRS_COR[0])-max(x.CRS_COR[1])

0.007319929742790943